## Load libraries

In [190]:
import pandas as pd # load and manipulate data
import numpy as np # calculation
import matplotlib.pyplot as plt # visualize data
from sklearn.preprocessing import OneHotEncoder # To one-hot encode variables
import category_encoders as ce # To target encode high cardinality variables

## Load raw dataset

In [191]:
demand = pd.read_csv('shopee_raw_data.csv')

## Overview

In [192]:
print(demand.head().to_string())

       date  product_id                                                                                    product_name     product_category    brand  traffic  impressions  payment  revenue  product_ad_spend  shop_ad_spend  auto_ad_spend  run_shop_ad  run_product_ad  product_page_bounce_count  traffic_from_search  wm_yr_wk  month  d  doubleday  near_dday  end_of_month  weekend  other_commercial_sale  day_offs  avg_price  promotion_on  promotion_price price_bin discount_rate  comment_received  product_rating  high_rating  high_discount  high_comment  avg_category_rate  avg_category_comment  week_of_month  wday  day_of_year
0  5/1/2024  3388329772  Combo 12 h?p cà phê viên nén Carraro  - Nh?p kh?u t? Ý - Tuong thích v?i máy capsule Nespresso  3. Carraro capsules  Carraro        0            0        0        0                 0             67          30000            1               1                          0                    0    202417      5  1          0          0             0

In [193]:
def check_unique_values(df):
    unique_values = {col: df[col].nunique() for col in df.columns}
    result_df = pd.DataFrame(list(unique_values.items()), columns=["Column Name", "Unique Value Count"])
    return result_df

check_unique_values(demand)

,Column Name,Unique Value Count
0,date,214
1,product_id,131
2,product_name,133
3,product_category,20
4,brand,13
5,traffic,56
6,impressions,97
7,payment,12
8,revenue,1185
9,product_ad_spend,567


In [194]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28676 entries, 0 to 28675
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       28676 non-null  object 
 1   product_id                 28676 non-null  int64  
 2   product_name               28676 non-null  object 
 3   product_category           28676 non-null  object 
 4   brand                      28676 non-null  object 
 5   traffic                    28676 non-null  int64  
 6   impressions                28676 non-null  int64  
 7   payment                    28676 non-null  int64  
 8   revenue                    28676 non-null  int64  
 9   product_ad_spend           28676 non-null  int64  
 10  shop_ad_spend              28676 non-null  int64  
 11  auto_ad_spend              28676 non-null  int64  
 12  run_shop_ad                28676 non-null  int64  
 13  run_product_ad             28676 non-null  int

Seems like we have some issues:
* Misalignment between `product_id` and `product_name`
* Some variables in the wrong data types

### Standardize data types

`date` should be in datetime data type

In [195]:
# Ensure the 'date' column is in datetime format
demand['date'] = pd.to_datetime(demand['date'])

# Sort the data by date, then product_id
demand = demand.sort_values(by=['product_id', 'date'])

`discount_rate` should be float   

`product_id` should be string

In [196]:
# Convert 'discount_rate' to float
demand['discount_rate'] = pd.to_numeric(
    demand['discount_rate'].str.replace('%', '', regex=False), 
    errors='coerce'
).astype(float).round(4)

# Convert 'product_id' to string
demand['product_id'] = demand['product_id'].astype(str)

Double-check

In [197]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28676 entries, 0 to 28675
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       28676 non-null  datetime64[ns]
 1   product_id                 28676 non-null  object        
 2   product_name               28676 non-null  object        
 3   product_category           28676 non-null  object        
 4   brand                      28676 non-null  object        
 5   traffic                    28676 non-null  int64         
 6   impressions                28676 non-null  int64         
 7   payment                    28676 non-null  int64         
 8   revenue                    28676 non-null  int64         
 9   product_ad_spend           28676 non-null  int64         
 10  shop_ad_spend              28676 non-null  int64         
 11  auto_ad_spend              28676 non-null  int64         
 12  run_

In [198]:
print(demand.head().to_string())

        date  product_id                                                                                    product_name     product_category    brand  traffic  impressions  payment  revenue  product_ad_spend  shop_ad_spend  auto_ad_spend  run_shop_ad  run_product_ad  product_page_bounce_count  traffic_from_search  wm_yr_wk  month  d  doubleday  near_dday  end_of_month  weekend  other_commercial_sale  day_offs  avg_price  promotion_on  promotion_price price_bin  discount_rate  comment_received  product_rating  high_rating  high_discount  high_comment  avg_category_rate  avg_category_comment  week_of_month  wday  day_of_year
0 2024-05-01  3388329772  Combo 12 h?p cà phê viên nén Carraro  - Nh?p kh?u t? Ý - Tuong thích v?i máy capsule Nespresso  3. Carraro capsules  Carraro        0            0        0        0                 0             67          30000            1               1                          0                    0    202417      5  1          0          0           

### Fix misalignment between `product_id` and `product_name`

In [199]:
def check_row_misalignment(df, col1, col2):
    misaligned = df.groupby(col1)[col2].nunique().reset_index()
    misaligned = misaligned[misaligned[col2] > 1]
    return misaligned

misaligned_rows = check_row_misalignment(demand, 'product_id', 'product_name')
print(misaligned_rows)

misaligned_product_ids = misaligned_rows['product_id'].tolist()

print("Check these product_ids for misalignment between product_id and product_name:")
print(misaligned_product_ids)

     product_id  product_name
43  15765129329             2
76  21382736254             2
81  22150796991             2
Check these product_ids for misalignment between product_id and product_name:
['15765129329', '21382736254', '22150796991']


In [200]:
# # I check it manually with excel to investigate, if you want to see for yourself, you can uncomment these lines

# filtered_by_product_id = demand[demand['product_id'].isin(misaligned_product_ids)]

# # Check the shape of the DataFrame
# rows, columns = filtered_by_product_id.shape

# print(f"Number of rows: {rows}")
# print(f"Number of columns: {columns}")

# # Export data to check in excel
# filtered_by_product_id.to_csv('filtered_by_product_id.csv')

I examined the data and see that there are dates where `product_name` A in `product_id` 1 had data. Within that same date, `product_name` B corresponding to the same `product_id` has 0 for its variables. And vice versa. So I'll keep rows where the `product_id` has "meaningful data" (non-zero values for its important variables), and eliminate rows where the `product_id` carries 0 for its variables (impressions, traffic, etc.) in another `product_name` for the same `date`.

This is because in November, the sales admin changed the content of the product pages: keeping high performing `product_id` (meaning the `product_id` contains high rating, high comment volumes, etc), merging low performing products into that `product_id` and then changed the name of the corresponding `product_name`.

In [201]:
# Filter the DataFrame for misaligned product_ids
filtered_demand = demand[demand['product_id'].isin(misaligned_product_ids)]

# Case 1: Remove duplicate rows based on the specified subset of columns
filtered_demand = filtered_demand.drop_duplicates(subset=['date', 'product_id', 'traffic', 'impressions', 'payment'])

# Case 2: After Case 1, check for each combination of date & product_id, if there are 2 rows,
# keep the row with impressions > 0 or payment > 0, else remove the row with impressions = 0.
def filter_impressions_and_payment(group):
    if len(group) == 2:  # Ensure there are exactly 2 rows per combination
        # Check for rows where impressions = 0 and the other row has either impressions > 0 or payment > 0
        if (group['impressions'] == 0).any() and ((group['impressions'] > 0).any() or (group['payment'] > 0).any()):
            # Keep the row where impressions > 0 or payment > 0
            group = group[(group['impressions'] > 0) | (group['payment'] > 0)]
    return group

# Apply the filtering logic only to the filtered DataFrame
filtered_demand = filtered_demand.groupby(['date', 'product_id'], group_keys=False).apply(filter_impressions_and_payment)

/tmp/ipykernel_4840/3250136266.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_demand = filtered_demand.groupby(['date', 'product_id'], group_keys=False).apply(filter_impressions_and_payment)


In [202]:
# Group by 'product_id' and count the number of rows for each
date_counts = filtered_demand.groupby('product_id')['date'].count()

# Set display option to show all rows
pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

# Print the total counts for each product_id
print("Total rows (all dates) for each product_id:")
print(date_counts)

Total rows (all dates) for each product_id:
product_id
15765129329    214
21382736254    214
22150796991    214
Name: date, dtype: int64


It's aligned now.

In [203]:
# Update the original DataFrame with the modified rows for misaligned product_ids
# First, drop the rows for the misaligned product_ids from the original DataFrame
demand = demand[~demand['product_id'].isin(misaligned_product_ids)]

# Then, append the filtered DataFrame back
demand = pd.concat([demand, filtered_demand], ignore_index=True)

## Feature engineering

### Ensure order of `date` for each `product_id`
Sorting will be applied first by `product_id` and then by `date`for creating lagging, and rolling features later

In [204]:
# Sort the data by product_id and date
demand = demand.sort_values(by=['product_id', 'date'])

demand.head()

,date,product_id,product_name,product_category,brand,traffic,impressions,payment,revenue,product_ad_spend,...,comment_received,product_rating,high_rating,high_discount,high_comment,avg_category_rate,avg_category_comment,week_of_month,wday,day_of_year
3424,2024-05-01,10070408774,Bình Ða D?ng 0.75L - Nh?p kh?u chính hãng 100%...,6. Other Melitta products,Melitta,0,0,0,0,0,...,0,0.0,0,0,0,4.9,8,1,3,122
3425,2024-05-02,10070408774,Bình Ða D?ng 0.75L - Nh?p kh?u chính hãng 100%...,6. Other Melitta products,Melitta,0,0,0,0,0,...,0,0.0,0,0,0,4.9,8,1,4,123
3426,2024-05-03,10070408774,Bình Ða D?ng 0.75L - Nh?p kh?u chính hãng 100%...,6. Other Melitta products,Melitta,0,0,0,0,0,...,0,0.0,0,0,0,4.9,8,1,5,124
3427,2024-05-04,10070408774,Bình Ða D?ng 0.75L - Nh?p kh?u chính hãng 100%...,6. Other Melitta products,Melitta,0,0,0,0,0,...,0,0.0,0,0,0,4.9,8,1,6,125
3428,2024-05-05,10070408774,Bình Ða D?ng 0.75L - Nh?p kh?u chính hãng 100%...,6. Other Melitta products,Melitta,0,0,0,0,0,...,0,0.0,0,0,0,4.9,8,1,7,126


### sine/cosine transforms time variables to capture periodicity and cyclic patterns

In [205]:
# For cyclical encoding, use sine/cosine transforms
def cyclical_encode(series, max_val):
    # series are values from 1 to max_val
    # Convert to 0-based: (value-1)
    sine = np.sin(2 * np.pi * (series - 1) / max_val)
    cosine = np.cos(2 * np.pi * (series - 1) / max_val)
    return sine, cosine

# Cyclical encode weekday (7 days in a week)
demand['wday_sin'], demand['wday_cos'] = cyclical_encode(demand['wday'], 7)
# Cyclical encode month (12 months)
demand['month_sin'], demand['month_cos'] = cyclical_encode(demand['month'], 12)
# Cyclical encode week_of_the_month (assume up to 5 weeks in a month)
demand['wom_sin'], demand['wom_cos'] = cyclical_encode(demand['week_of_month'], 5)
# Cyclical encode day_of_year (assume up to 365 days in a year)
demand['day_of_year_sin'], demand['day_of_year_cos'] = cyclical_encode(demand['day_of_year'], 365)

### Lag and rolling features for important numeric columns
How I defined 'important' is based on domain knowledge and intuition first, then we can refine features later

In [206]:
# Define the columns to create the features on
numeric_cols = ['payment', 'revenue', 'product_ad_spend', 'shop_ad_spend', 'auto_ad_spend']

# Create lag features (e.g., lag_1, lag_3, lag_7, lag_14, lag_28)
lag_periods = [3, 28]
for col in numeric_cols:
    if col in demand.columns:
        for lag in lag_periods:
            demand[f'{col}_lag_{lag}'] = demand.groupby('product_id')[col].shift(lag) #Reset lagging for every product_id

### Create rolling features (example: 7-day rolling mean)

In [207]:
# Define the columns to create the features on
numeric_cols = ['traffic', 'impressions', 'payment', 'revenue', 'product_ad_spend', 'shop_ad_spend', 'auto_ad_spend']

rolling_windows = [7, 28]  # Define the rolling windows
for rolling_window in rolling_windows:
    for col in numeric_cols:  # Ensure these are the numeric columns you want to apply rolling features to
        if col in demand.columns:
            demand[f'{col}_rolling_{rolling_window}d_mean'] = (
                demand.groupby('product_id')[col] #Reset rolling for every product_id
                .transform(lambda x: x.shift(1).rolling(rolling_window).mean())
            )

### Category-level average within each month and each week
* There are homogenous patterns for each `product_category` (reference 'M5 Competition'), so we need to extract category-level data for the model to enhance learning process.
* Besides, in execution, it is easier to manage on category level, not on every single product level. So if we can learn anything helpful, we can apply into our category growth plan.

In [208]:
if 'product_category' in demand.columns:
    # Daily aggregation
    cat_rev = demand.groupby(['product_category', 'date'])['product_ad_spend'].mean().reset_index(name='cat_daily_avg_product_ad_spend')
    cat_traffic = demand.groupby(['product_category', 'date'])['traffic'].mean().reset_index(name='cat_daily_avg_traffic')

    # Monthly aggregation
    cat_month_rev = demand.groupby(['product_category', 'month'])['product_ad_spend'].mean().reset_index(name='cat_month_avg_product_ad_spend')
    cat_month_traffic = demand.groupby(['product_category', 'month'])['traffic'].mean().reset_index(name='cat_month_avg_traffic')

    # Weekly aggregation (assuming 'week_of_month' corresponds to the day of the week)
    cat_week_rev = demand.groupby(['product_category', 'week_of_month'])['product_ad_spend'].mean().reset_index(name='cat_week_avg_product_ad_spend')
    cat_week_traffic = demand.groupby(['product_category', 'week_of_month'])['traffic'].mean().reset_index(name='cat_week_avg_traffic')

    # Merging daily aggregated values back to the main dataframe
    demand = pd.merge(demand, cat_rev, on=['product_category', 'date'], how='left')
    demand = pd.merge(demand, cat_traffic, on=['product_category', 'date'], how='left')

    # Merging monthly aggregated values
    demand = pd.merge(demand, cat_month_rev, on=['product_category', 'month'], how='left')
    demand = pd.merge(demand, cat_month_traffic, on=['product_category', 'month'], how='left')

    # Merging weekly aggregated values
    demand = pd.merge(demand, cat_week_rev, on=['product_category', 'week_of_month'], how='left')
    demand = pd.merge(demand, cat_week_traffic, on=['product_category', 'week_of_month'], how='left')

#### Continuous zero-sale days until today
I learned about `continuous_zero_sales_days` from the repetitive feature engineering practices in the M5 competition. I believe this feature is highly effective in helping the model understand sporadic sales patterns for certain `product_id`s and `product_category`s.

In [211]:
# Sort data to ensure it is in time order
demand = demand.sort_values(by=['product_id', 'date'])

# Define continuous zero-sale days
def continuous_zero_sales(group):
    zero_sales_days = (group['payment'] == 0).astype(int)  # 1 if no sale, 0 otherwise
    return zero_sales_days.groupby((zero_sales_days != zero_sales_days.shift()).cumsum()).cumsum()

# Apply the function for each product_id
demand['continuous_zero_sales_days'] = (
    demand.groupby('product_id', group_keys=False).apply(continuous_zero_sales)
)

/tmp/ipykernel_4840/269506180.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  demand.groupby('product_id', group_keys=False).apply(continuous_zero_sales)


Since this feature is important, to create lagging and rolling features for it is not a bad idea, I reckon

In [213]:
# Rolling features for `continuous_zero_sales_days` on product_id level
# Rolling window of 14 days is based on domain knowledge and reviewing the dataset
rolling_window = 14
demand['zero_sales_rolling_mean_14d'] = (
    demand.groupby('product_id')['continuous_zero_sales_days']
          .transform(lambda x: x.rolling(rolling_window, min_periods=1).mean())
)
demand['zero_sales_rolling_max_14d'] = (
    demand.groupby('product_id')['continuous_zero_sales_days']
          .transform(lambda x: x.rolling(rolling_window, min_periods=1).max())
)

In [214]:
# Category level zero_sales_rolling_mean_14d
# Compute category-level average of zero_sales_rolling_mean_14d
cat_rolling_avg_zero_sales_14d = (
    demand.groupby(['product_category', 'date'])['zero_sales_rolling_mean_14d']
          .mean()
          .reset_index()
          .rename(columns={'zero_sales_rolling_mean_14d': 'cat_rolling_avg_zero_sales_14d'})
)

# Merge back to the original DataFrame
demand = pd.merge(demand, cat_rolling_avg_zero_sales_14d, on=['product_category', 'date'], how='left')

In [215]:
# Interaction between category-level rolling zero sales and promotion
demand['cat_zero_sales_x_promotion'] = (
    demand['cat_rolling_avg_zero_sales_14d'] * demand['promotion_on']
)

In [216]:
# Step 0: Ensure data is sorted by 'product_category' and 'date'
demand = demand.sort_values(by=['product_category', 'date'])

# Trend in rolling average of `continuous_zero_sales_days`
# Step 1: Compute category-level trend of zero_sales_rolling_mean_14d
# Ensure it considers time order properly
cat_trend = (
    demand.groupby(['product_category', 'date'])['cat_rolling_avg_zero_sales_14d']
          .mean()  # Aggregate for consistent value per 'product_category' and 'date'
          .groupby(level=0)  # Group again by 'product_category'
          .transform(lambda x: x.diff(14))  # Calculate 14-day trend
          .reset_index(name='cat_rolling_avg_zero_sales_trend_14d')  # Create a new DataFrame
)

# Step 2: Merge back to the original DataFrame on 'product_category' and 'date'
demand = pd.merge(demand, cat_trend, on=['product_category', 'date'], how='left')

# Percentage changes in rolling average of `continuous_zero_sales_days`
# Step 1: Compute category-level trend of zero_sales_rolling_mean_14d
# Ensure it considers time order properly
cat_pect_trend = (
    demand.groupby(['product_category', 'date'])['cat_rolling_avg_zero_sales_14d']
          .mean()  # Aggregate for consistent value per 'product_category' and 'date'
          .groupby(level=0)  # Group again by 'product_category'
          .transform(lambda x: x.pct_change(14))  # Calculate 14-day trend
          .reset_index(name='cat_rolling_avg_zero_sales_pct_change_14d')  # Create a new DataFrame
)

# Step 2: Merge back to the original DataFrame on 'product_category' and 'date'
demand = pd.merge(demand, cat_pect_trend, on=['product_category', 'date'], how='left')

In [217]:
# Sort data to ensure it is in time order
demand = demand.sort_values(by=['date', 'product_id'])

# Lagged values of category-level zero sales rolling average
for lag in [7, 14, 28]:
    demand[f'cat_rolling_avg_zero_sales_lag_{lag}d'] = (
        demand.groupby('product_id')['cat_rolling_avg_zero_sales_14d']
              .transform(lambda x: x.shift(lag))
    )

## Normalized price-related variables:
* Scale: category level

In [218]:
# Step 1: Group by 'product_category' and calculate mean and std for 'avg_price' and 'promotion_price'
category_stats = demand.groupby('product_category').agg(
    avg_price_mean=('avg_price', 'mean'),
    avg_price_std=('avg_price', 'std'),
    promotion_price_mean=('promotion_price', 'mean'),
    promotion_price_std=('promotion_price', 'std')
).reset_index()

# Step 2: Merge these statistics back to the original DataFrame
demand = pd.merge(demand, category_stats, on='product_category', how='left')

# Step 3: Create normalized columns
demand['normalized_avg_price'] = (demand['avg_price'] - demand['avg_price_mean']) / demand['avg_price_std']
demand['normalized_promotion_price'] = (demand['promotion_price'] - demand['promotion_price_mean']) / demand['promotion_price_std']

# Dropping unnecessary intermediate columns for cleanliness (optional)
demand = demand.drop(columns=['avg_price_mean', 'avg_price_std', 'promotion_price_mean', 'promotion_price_std'])

### Features based on domain knowledge

In [209]:
# Ad spend during promotion, weekend
if 'promotion_on' in demand.columns and 'product_ad_spend' in demand.columns:
    demand['ad_spend_during_promotion'] = demand['product_ad_spend'] * demand['promotion_on']

if 'weekend' in demand.columns and 'product_ad_spend' in demand.columns:
    demand['weekend_ad_spend'] = demand['weekend'] * demand['product_ad_spend']

In [210]:
# CTR
if 'traffic' in demand.columns and 'impressions' in demand.columns:
    demand['CTR'] = np.where(demand['impressions'] == 0, 0, demand['traffic'] / demand['impressions'])

### Last steps
## One-hot encoding

In [219]:
# Identify categorical features for one-hot encoding
cat_cols = []
for col in ['product_category', 'brand', 'price_bin']:
    if col in demand.columns and demand[col].dtype == 'object':
        cat_cols.append(col)

# Perform one-hot encoding
if cat_cols:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # Use sparse_output instead of sparse
    # Fit-transform
    ohe_data = ohe.fit_transform(demand[cat_cols])
    # Create a DataFrame with OHE results
    ohe_df = pd.DataFrame(ohe_data, columns=ohe.get_feature_names_out(cat_cols), index=demand.index)
    # Concatenate back
    demand = pd.concat([demand.drop(cat_cols, axis=1), ohe_df], axis=1)

Due to the high cardinality of `product_id`, one-hot encoding is not a suitable approach.

So I'll try target encoding on it.

## Target encoding: product_id

In [220]:
#Instantiate a target encoder
te = ce.TargetEncoder(cols=['product_id'])

# Fit the encoder on the training set
demand['product_id_target_enc'] = te.fit_transform(demand['product_id'], demand['payment'])

# Drop original if desired
demand.drop('product_id', axis=1, inplace=True)

## Drop columns to avoid duplication in learning

In [222]:
demand = demand.drop(['wday', 'month', 'week_of_month', 'product_name', 'day_of_year'], axis=1)